<a href="https://colab.research.google.com/github/MarcoParola/medical_images_classification/blob/main/Scratch_CNN_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import collections 

In [ ]:
projectPath = 'drive/MyDrive/Colab Notebooks/cidl/'
dataPath = 'numpy data/'

In [ ]:
%cd $projectPath
%run utils.ipynb
%cd '../../../..'

In [ ]:
# load data
imagesTrainPub, labelsTrainPub, imagesTestPub, labelsTestPub, imagesTestPriv = load_data(projectPath + dataPath)

# normalize data 
imagesTrainPub = scaleData(imagesTrainPub)
imagesTestPub = scaleData(imagesTestPub)

In [ ]:
# create (if not exists) the folder to store the models
dir_name = 'models'
if not os.path.exists(projectPath + dir_name):
  os.makedirs(projectPath + dir_name)

# **Task 2**

## 2.2 Binary calssification: Benign and Malignant
Classes:
*   Benign 1568
*   Malignant 1108



In [ ]:
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

# 0 -> benign
# 1 -> malignant

# prepare trainig-set
trainingSet = []
trainingLabels = []

for i in range(len(labelsTrainPub)):
  if labelsTrainPub[i] == 1 or labelsTrainPub[i] == 3:
    trainingSet.append(imagesTrainPub[i])
    trainingLabels.append(0)
  if labelsTrainPub[i] == 2 or labelsTrainPub[i] == 4:
    trainingSet.append(imagesTrainPub[i])
    trainingLabels.append(1)

trainingSet = np.array(trainingSet)
trainingSet = trainingSet.reshape(2676, 150, 150, 1)


training = []
for i in range(len(trainingSet)):
  training.append((trainingSet[i], trainingLabels[i]))

training = np.array(training)
np.random.shuffle(training)


trainingSet = []
trainingLabels = []

for i in range(len(training)):
  trainingSet.append(training[i][0])
  trainingLabels.append(training[i][1])

trainingSet = np.array(trainingSet)
trainingLabels = np.array(trainingLabels)

# prepare test-set
testSet = []
testLabels = []

for i in range(len(labelsTestPub)):
  if labelsTestPub[i] == 1 or labelsTestPub[i] == 3:
    testSet.append(imagesTestPub[i])
    testLabels.append(0)
  if labelsTestPub[i] == 2 or labelsTestPub[i] == 4:
    testSet.append(imagesTestPub[i])
    testLabels.append(1)

testSet = np.array(testSet)
testSet = testSet.reshape(336, 150, 150, 1)

In [ ]:
plt.hist(trainingLabels, bins=np.arange(trainingLabels.min()-0.25, trainingLabels.max()+1), width=0.5)
plt.xticks(np.arange(trainingLabels.min(), trainingLabels.max()+1, 1.0))
plt.show()
collections.Counter(trainingLabels)

In [ ]:
trainingLabels = to_categorical(np.array(trainingLabels))
testLabels = to_categorical(np.array(testLabels))

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models


model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1), padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
earlyStopCallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

# TODO
#earlyStopCallback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', min_delta=1)

#model.fit(trainingSet, trainingLabels, epochs=26, batch_size=64)
#model.fit(trainingSet, trainingLabels, epochs=100, callbacks=[earlyStopCallback], batch_size=32, validation_data=(testSet, testLabels))
model.fit(trainingSet, trainingLabels, epochs=100, callbacks=[earlyStopCallback], batch_size=32, validation_split=0.2)

#model.fit(trainingSet, trainingLabels, validation_split=0.2, batch_size=32)


In [ ]:
test_loss, test_acc = model.evaluate(testSet, testLabels)
print(test_loss, test_acc)

In [ ]:


train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

print(len(trainingSet[0]))

x = train_datagen.flow(trainingSet, trainingLabels, shuffle=True)
print(len(x[0]))



In [ ]:
trainingLabels[0]

In [ ]:
'''
# 
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1), padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))


model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()
'''

In [ ]:
tf.random.set_seed(18)

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1), padding='same'))
model.add(layers.Dropout(rate=.25))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.Dropout(rate=.25))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.Dropout(rate=.25))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.Dropout(rate=.25))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))


model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
test_loss, test_acc = model.evaluate(testSet, testLabels)